In [86]:
from data import Data
from dimension_reduction import PCADimensionReduction
from autoencoder import Autoencoder

from keras.models import Sequential
from keras.layers import Dense

In [87]:
data_filepath = "data/SC_integration/counts_ctc_simulated_123_5k.tsv"
true_results_filepath = "data/SC_integration/ids_ctc_simulated_123_5k.tsv"
healthy_train_indices_filepath = "data/SC_integration/healthy_train_indices.npy"
healthy_test_indices_filepath = "data/SC_integration/healthy_test_indices.npy"

SEED = 42
FOLD_NUMBER = 3

# You can change these values to work better for models
CUT_BY_MAX_THRESHOLD = 4
PCA_VARIABLES_AMOUNT = 60

In [88]:
data_object = Data(data_filepath, true_results_filepath)

healthy_train_data, healthy_test_data, cancer_data =  data_object.get_train_test_healthy_split_cancer_data(healthy_train_indices_filepath, healthy_test_indices_filepath)
scaled_healthy_train_data, scaled_healthy_test_data, scaled_cancer_data = data_object.get_scaled_healthy_train_test_cancer_data()

In [89]:
cut_by_max_healthy_train_data, cut_by_max_healthy_test_data, cut_by_max_cancer_data = data_object.get_cut_by_max_healthy_train_test_cancer_data(CUT_BY_MAX_THRESHOLD)

pca_object = PCADimensionReduction(cut_by_max_healthy_train_data, scaled_healthy_train_data, data_object.train_true_results, SEED)
pca_variables = pca_object.get_most_important_variables_from_pc1(PCA_VARIABLES_AMOUNT)
pca_reduced_healthy_train_data = cut_by_max_healthy_train_data[pca_variables.index]
pca_reduced_healthy_test_data = cut_by_max_healthy_test_data[pca_variables.index]
pca_reduced_cancer_data = cut_by_max_cancer_data[pca_variables.index]

In [90]:
def create_autoencoder(input_shape):
    model = Sequential()
    model.add(Dense(25, input_dim=input_shape, activation='relu'))
    model.add(Dense(5, activation='relu')) # size to compress to
    model.add(Dense(25, activation='relu'))
    model.add(Dense(input_shape))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

## Autoencoder

### Regular data

In [91]:
autoencoder = create_autoencoder(healthy_train_data.shape[1])
autoencoder.fit(healthy_train_data, healthy_train_data, batch_size=16, epochs=100)

Epoch 1/100
190/190 [==============================] - 0s 1ms/step - loss: 0.1763
Epoch 2/100
190/190 [==============================] - 0s 1ms/step - loss: 0.1422
Epoch 3/100
190/190 [==============================] - 0s 985us/step - loss: 0.1416
Epoch 4/100
190/190 [==============================] - 0s 980us/step - loss: 0.1412
Epoch 5/100
190/190 [==============================] - 0s 1ms/step - loss: 0.1407
Epoch 6/100
190/190 [==============================] - 0s 1ms/step - loss: 0.1402
Epoch 7/100
190/190 [==============================] - 0s 999us/step - loss: 0.1398
Epoch 8/100
190/190 [==============================] - 0s 1ms/step - loss: 0.1393
Epoch 9/100
190/190 [==============================] - 0s 1ms/step - loss: 0.1388
Epoch 10/100
190/190 [==============================] - 0s 981us/step - loss: 0.1379
Epoch 11/100
190/190 [==============================] - 0s 988us/step - loss: 0.1369
Epoch 12/100
190/190 [==============================] - 0s 987us/step - loss: 0.1354
E

In [92]:
autoencoder_object = Autoencoder(autoencoder, healthy_train_data, healthy_test_data, cancer_data)
autoencoder_object.display_average_rmse()
print(f"Threshold {autoencoder_object.calculate_threshold()}")
autoencoder_object.display_classification_metrics()

2/2 [==============================] - 0s 1ms/step
Train RMSE: 0.33606304929540515
Test RMSE: 0.33864032841435676
Cancer data RMSE: 0.5047450235386485
Threshold 0.4216926759765026
Train accuracy: 0.9674234945705824
Test accuracy: 0.9524174980813507
Cancer accuracy: 0.98


### Scaled data

In [93]:
autoencoder = create_autoencoder(scaled_healthy_train_data.shape[1])
autoencoder.fit(scaled_healthy_train_data, scaled_healthy_train_data, batch_size=16, epochs=100)

Epoch 1/100
190/190 [==============================] - 0s 993us/step - loss: 0.9362
Epoch 2/100
190/190 [==============================] - 0s 987us/step - loss: 0.9146
Epoch 3/100
190/190 [==============================] - 0s 1ms/step - loss: 0.9084
Epoch 4/100
190/190 [==============================] - 0s 997us/step - loss: 0.9036
Epoch 5/100
190/190 [==============================] - 0s 984us/step - loss: 0.9002
Epoch 6/100
190/190 [==============================] - 0s 998us/step - loss: 0.8971
Epoch 7/100
190/190 [==============================] - 0s 986us/step - loss: 0.8947
Epoch 8/100
190/190 [==============================] - 0s 1ms/step - loss: 0.8926
Epoch 9/100
190/190 [==============================] - 0s 993us/step - loss: 0.8910
Epoch 10/100
190/190 [==============================] - 0s 981us/step - loss: 0.8894
Epoch 11/100
190/190 [==============================] - 0s 992us/step - loss: 0.8879
Epoch 12/100
190/190 [==============================] - 0s 987us/step - loss: 

In [94]:
autoencoder_object = Autoencoder(autoencoder, scaled_healthy_train_data, scaled_healthy_test_data, scaled_cancer_data)
autoencoder_object.display_average_rmse()
print(f"Threshold {autoencoder_object.calculate_threshold()}")
autoencoder_object.display_classification_metrics()

2/2 [==============================] - 0s 1ms/step
Train RMSE: 0.9068982602365688
Test RMSE: 0.9094301803059419
Cancer data RMSE: 6.340537650851735
Threshold 3.6249839155788384
Train accuracy: 1.0
Test accuracy: 0.9984650805832693
Cancer accuracy: 0.98


### Cut by max data

In [95]:
autoencoder = create_autoencoder(cut_by_max_healthy_train_data.shape[1])
autoencoder.fit(cut_by_max_healthy_train_data, cut_by_max_healthy_train_data, batch_size=16, epochs=100)

Epoch 1/100
190/190 [==============================] - 0s 917us/step - loss: 2.2233
Epoch 2/100
190/190 [==============================] - 0s 914us/step - loss: 0.6577
Epoch 3/100
190/190 [==============================] - 0s 919us/step - loss: 0.5445
Epoch 4/100
190/190 [==============================] - 0s 920us/step - loss: 0.5343
Epoch 5/100
190/190 [==============================] - 0s 909us/step - loss: 0.5303
Epoch 6/100
190/190 [==============================] - 0s 919us/step - loss: 0.5276
Epoch 7/100
190/190 [==============================] - 0s 921us/step - loss: 0.5242
Epoch 8/100
190/190 [==============================] - 0s 917us/step - loss: 0.5099
Epoch 9/100
190/190 [==============================] - 0s 912us/step - loss: 0.4846
Epoch 10/100
190/190 [==============================] - 0s 912us/step - loss: 0.4725
Epoch 11/100
190/190 [==============================] - 0s 909us/step - loss: 0.4671
Epoch 12/100
190/190 [==============================] - 0s 916us/step - lo

In [96]:
autoencoder_object = Autoencoder(autoencoder, cut_by_max_healthy_train_data, cut_by_max_healthy_test_data, cut_by_max_cancer_data)
autoencoder_object.display_average_rmse()
print(f"Threshold {autoencoder_object.calculate_threshold()}")
autoencoder_object.display_classification_metrics()

2/2 [==============================] - 0s 1ms/step
Train RMSE: 0.6175189016040502
Test RMSE: 0.6170845385182439
Cancer data RMSE: 0.8182444980667277
Threshold 0.717881699835389
Train accuracy: 0.8269167489305692
Test accuracy: 0.8188795088257866
Cancer accuracy: 0.7


### PCA reduced

In [97]:
autoencoder = create_autoencoder(pca_reduced_healthy_train_data.shape[1])
autoencoder.fit(pca_reduced_healthy_train_data, pca_reduced_healthy_train_data, batch_size=16, epochs=100)

Epoch 1/100
190/190 [==============================] - 0s 902us/step - loss: 2.6439
Epoch 2/100
190/190 [==============================] - 0s 911us/step - loss: 0.6718
Epoch 3/100
190/190 [==============================] - 0s 913us/step - loss: 0.5830
Epoch 4/100
190/190 [==============================] - 0s 901us/step - loss: 0.5632
Epoch 5/100
190/190 [==============================] - 0s 914us/step - loss: 0.5296
Epoch 6/100
190/190 [==============================] - 0s 912us/step - loss: 0.5077
Epoch 7/100
190/190 [==============================] - 0s 951us/step - loss: 0.4980
Epoch 8/100
190/190 [==============================] - 0s 945us/step - loss: 0.4923
Epoch 9/100
190/190 [==============================] - 0s 961us/step - loss: 0.4866
Epoch 10/100
190/190 [==============================] - 0s 947us/step - loss: 0.4836
Epoch 11/100
190/190 [==============================] - 0s 918us/step - loss: 0.4815
Epoch 12/100
190/190 [==============================] - 0s 913us/step - lo

In [98]:
autoencoder_object = Autoencoder(autoencoder, pca_reduced_healthy_train_data, pca_reduced_healthy_test_data, pca_reduced_cancer_data)
autoencoder_object.display_average_rmse()
print(f"Threshold {autoencoder_object.calculate_threshold()}")
autoencoder_object.display_classification_metrics()

2/2 [==============================] - 0s 1ms/step
Train RMSE: 0.6423351878589655
Test RMSE: 0.6411983500285833
Cancer data RMSE: 0.836493613487297
Threshold 0.7394144006731312
Train accuracy: 0.8081605791378743
Test accuracy: 0.8104374520337683
Cancer accuracy: 0.7
